# Finetuning of the pretrained Japanese BERT model

Finetune the pretrained model to solve multi-class classification problems.  
This notebook requires the following objects:
- trained sentencepiece model (model and vocab files)
- pretraiend Japanese BERT model

We make test:dev:train = 2:2:6 datasets.

## Data preparing


Data preprocessing.
dev,train,testに分ける処理は一旦さておく

In [ ]:
df = pd.DataFrame({'text' : all_text, 'label' : all_label})

In [ ]:
df = df.sample(frac=1, random_state=23).reset_index(drop=True)

Save data as tsv files.  
test:dev:train = 2:2:6. To check the usability of finetuning, we also prepare sampled training data (1/5 of full training data).

In [ ]:
df[:len(df) // 5].to_csv( os.path.join(EXTRACTDIR, "test.tsv"), sep='\t', index=False)
df[len(df) // 5:len(df)*2 // 5].to_csv( os.path.join(EXTRACTDIR, "dev.tsv"), sep='\t', index=False)
df[len(df)*2 // 5:].to_csv( os.path.join(EXTRACTDIR, "train.tsv"), sep='\t', index=False)

### 1/5 of full training data.
# df[:len(df) // 5].to_csv( os.path.join(EXTRACTDIR, "test.tsv"), sep='\t', index=False)
# df[len(df) // 5:len(df)*2 // 5].to_csv( os.path.join(EXTRACTDIR, "dev.tsv"), sep='\t', index=False)
# df[len(df)*2 // 5:].sample(frac=0.2, random_state=23).to_csv( os.path.join(EXTRACTDIR, "train.tsv"), sep='\t', index=False)

この時点で、指定フォルダにtrain, dev, test tsvを○○フォルダに配置する

## Finetune pre-trained model

It will take a lot of hours to execute the following cells on CPU environment.  

In [ ]:
PRETRAINED_MODEL_PATH = '../model/model.ckpt-1400000'
FINETUNE_OUTPUT_DIR = '../model/livedoor_output'

DATA_DIR='../data'

In [ ]:
%%time
# It will take many hours on CPU environment.

!python3 ../src/run_classifier.py \
  --do_train=true \
  --do_eval=true \
  --data_dir={DATA_DIR} \
  --model_file=../model/wiki-ja.model \
  --vocab_file=../model/wiki-ja.vocab \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --max_seq_length=512 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=10 \
  --output_dir={FINETUNE_OUTPUT_DIR}

## Predict using the finetuned model

Let's predict test data using the finetuned model.  

In [ ]:
%%time
# It will take many hours on CPU environment.

!python3 ../src/run_classifier.py \
  --do_predict=true \
  --data_dir=../data/livedoor \
  --model_file=../model/wiki-ja.model \
  --vocab_file=../model/wiki-ja.vocab \
  --output_dir={FINETUNE_OUTPUT_DIR}

In [ ]:
import sys
sys.path.append("../src")

from run_classifier import LivedoorProcessor

In [ ]:
processor = LivedoorProcessor()
label_list = processor.get_labels()

In [ ]:
result = pd.read_csv(FINETUNE_OUTPUT_DIR+"/test_results.tsv", sep='\t', header=None)

In [ ]:
result.head()

Read test data set and add prediction results.

In [ ]:
import pandas as pd

In [ ]:
test_df = pd.read_csv("../data/livedoor/test.tsv", sep='\t')

In [ ]:
test_df['predict'] = [ label_list[idx] for idx in result.idxmax(axis=1) ]

In [ ]:
test_df.head()

In [ ]:
sum( test_df['label'] == test_df['predict'] ) / len(test_df)

A littel more detailed check using `sklearn.metrics`.

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
print(classification_report(test_df['label'], test_df['predict']))

In [ ]:
print(confusion_matrix(test_df['label'], test_df['predict']))

### Simple baseline model.

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
train_df = pd.read_csv("../data/livedoor/train.tsv", sep='\t')
dev_df = pd.read_csv("../data/livedoor/dev.tsv", sep='\t')
test_df = pd.read_csv("../data/livedoor/test.tsv", sep='\t')

In [ ]:
!apt-get install -q -y mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8

In [ ]:
!pip install mecab-python3==0.7

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
import MeCab

In [ ]:
m = MeCab.Tagger("-Owakati")

In [ ]:
train_dev_df = pd.concat([train_df, dev_df])

In [ ]:
train_dev_xs = train_dev_df['text'].apply(lambda x: m.parse(x))
train_dev_ys = train_dev_df['label']

test_xs = test_df['text'].apply(lambda x: m.parse(x))
test_ys = test_df['label']

In [ ]:
vectorizer = TfidfVectorizer(max_features=750)
train_dev_xs_ = vectorizer.fit_transform(train_dev_xs)
test_xs_ = vectorizer.transform(test_xs)

The following set up is not exactly identical to that of BERT because inside Classifier it uses `train_test_split` with shuffle.  
In addition, parameters are not well tuned, however, we think it's enough to check the power of BERT.

In [ ]:
%%time

model = GradientBoostingClassifier(n_estimators=200,
                                   validation_fraction=len(dev_df)/len(train_df),
                                   n_iter_no_change=5,
                                   tol=0.01,
                                   random_state=23)

### 1/5 of full training data.
# model = GradientBoostingClassifier(n_estimators=200,
#                                    validation_fraction=len(dev_df)/len(train_df),
#                                    n_iter_no_change=5,
#                                    tol=0.01,
#                                    random_state=23)

model.fit(train_dev_xs_, train_dev_ys)

In [ ]:
print(classification_report(test_ys, model.predict(test_xs_)))

In [ ]:
print(confusion_matrix(test_ys, model.predict(test_xs_)))